# Aprendizado Supervisionado no Neurônio Perceptron

In [ ]:
# Importando as ferramentas

import numpy as np
import matplotlib.pyplot as plt
import random as rd
from sklearn import metrics
import math

##  Parte I– Resolvendo um Problema Linearmente Separável


1. Usar o arquivo dataAll.txt
2. Construir o algoritmo de treinamento do neurônio perceptron.

In [ ]:
arq_input = "dataAll.txt"

dataAll = np.fromfile(arq_input, dtype=np.float64)
print("Tipo de dataAll: {}".format(type(dataAll)))
print("Número de elementos em dataAll: {}".format(dataAll.size))
print("\n")


# -1 infers the size of the new dimension from the size of the input array.
dataAll = np.reshape(dataAll, (-1, 3))
print(dataAll[:10]) # Mostrando 10 exemplos

print("\n")
print("Shape de dataAll: {}".format(dataAll.shape))
print("Dimensões de dataAll: {}".format(dataAll.ndim))

Como havia 3000 exemplos anteriormente e agora a matriz tem dimensões (1, 3). 
Então, há 3000/3 = 1000 exemplos em dataAll. Ou seja, (1000, 1, 3).

In [ ]:
# Separando em x, y e rotulo 

print("x:")
x = dataAll[:,:2]
print(x[:10]) # Mostrando os 10 primeiros pontos 
print(x[-1])
print("\n")

print("y:")
y = dataAll[:,2:].astype(int)
print(y[:10]) # Rótulos dos pontos

Criando o neurônio Perceptron:

In [65]:
class Perceptron:
    """
    A perceptron is a classification model that consists of a set of weights, or scores,
    one for every feature, and a threshold. The perceptron multiplies each weight by its
    corresponding score, and adds them, obtaining a score.
    by Rajwrite Nath on Medium
    
    """

    def __init__(self, threshold, lr):
        self.threshold = threshold
        self.lr = lr
        self.weights = None
        self.scores = None
        self.adjustments = 0
        self.epochs = 0

    def gen_weights(self, low, high, size):
        self.weights = np.random.uniform(low, high, size).reshape(1000, 2)

    def step_fn(self, z):
        return np.where(z >= self.threshold, 1, 0)

    def error_fn(self, y_true, y_predicted):
        return np.sum(y_true - y_predicted)

    def predict(self, x_data):
        return self.step_fn(np.sum(x_data*self.weights, axis=1).reshape(x_data.shape[0], -1))

    def fit(self, x_data, y_data):
        self.gen_weights(-0.5, 0.5, size=x_data.size)
        print("Início: {}".format(self.weights))
        
        y_predicted = self.predict(x_data)
        self.scores = y_predicted  # Atualiza os scores
        
        # Algoritmo executa até a convergência, supomos que as classes sejam linearmente separáveis
        while not np.all(y_data == self.scores):
            
            self.epochs += 1  # Não convergiu, vai necessitar de mais 1 época
            
            # Para os pesos que resultaram em uma predição errada:
            
            predicoes_incorretas = (y_data != self.scores)
            
            for i in np.where(predicoes_incorretas)[0]:
                x_i = x_data[i]
                y_i = y_data[i]
                y_pred = y_predicted[i]

                # Calculando erros
                error = self.error_fn(y_i, y_pred)

                # Relculando pesos
                self.weights += self.lr * error * x_i
                self.adjustments += 1
                
            y_predicted = self.predict(x_data) 
            self.scores = y_predicted # Atualiza os scores novamente com base nos ajustes

# Inicializando a semente
np.random.seed(0)

# Inicializando o neurônio
neuronio_perceptron = Perceptron(threshold=0, lr=0.1)
neuronio_perceptron.fit(x, y)
print("Final: {}".format(neuronio_perceptron.weights))

Início: [[ 0.0488135   0.21518937]
 [ 0.10276338  0.04488318]
 [-0.0763452   0.14589411]
 ...
 [-0.08556113  0.29128155]
 [ 0.22119811 -0.01989219]
 [ 0.14386404  0.00177313]]
Final: [[ 16137.7740335  -16151.37531063]
 [ 16137.82798338 -16151.54561682]
 [ 16137.6488748  -16151.44460589]
 ...
 [ 16137.63965887 -16151.29921845]
 [ 16137.94641811 -16151.61039219]
 [ 16137.86908404 -16151.58872687]]


In [ ]:
# Gerando o gráfico
    
def gen_grafico(x, y, data_x, title, final_w):
    
    peso_x1 = final_w[-1, 0]
    peso_x2 = final_w[-1, 1]
    
    
    fig, ax = plt.subplots()

    cor = {0: 'red', 1:'blue'}
    for i in range(x.shape[0]):
        plt.plot(x[i][0], x[i][1], marker='o', linestyle='', markersize=1, c = cor[y[i][0]])
    
    reta = (final_w[:][0] / final_w[:][0]) - (final_w[:][1] / final_w[:][2]) * data[:][1]
    plt.plot(reta, color='purple', linestyle='--')
    
    ax.legend(cor, loc='center left', bbox_to_anchor=(1, 0.5))
    ax.grid(True)
    ax.set_title(title)
    plt.show()
             
gen_grafico(x, y, "Distribução dos exemplos em dataAll", neuronio_perceptron.weights)             